<a href="https://colab.research.google.com/github/AnjaliKumari1234567/Image-Captioning-Model/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("Hello")

Hello


In [ ]:
print("Hi there!")

Hi there!
